<a href="https://colab.research.google.com/github/elichen/aoc2017/blob/main/Day_18_Duet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
data = """snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d
""".split('\n')

In [78]:
data = """set i 31
set a 1
mul p 17
jgz p p
mul a 2
add i -1
jgz i -2
add a -1
set i 127
set p 618
mul p 8505
mod p a
mul p 129749
add p 12345
mod p a
set b p
mod b 10000
snd b
add i -1
jgz i -9
jgz a 3
rcv b
jgz b -1
set f 0
set i 126
rcv a
rcv b
set p a
mul p -1
add p b
jgz p 4
snd a
set a b
jgz 1 3
snd b
set f 1
add i -1
jgz i -11
snd a
jgz f -16
jgz a -19""".split('\n')

In [79]:
class SoundMachine:
    def __init__(self):
        self.registers = {}
        self.last_sound = None
        self.instruction_pointer = 0

    def get_value(self, operand):
        try:
            return int(operand)
        except ValueError:
            return self.registers.get(operand, 0)

    def snd(self, X):
        self.last_sound = self.get_value(X)

    def set(self, X, Y):
        self.registers[X] = self.get_value(Y)

    def add(self, X, Y):
        self.registers[X] = self.get_value(X) + self.get_value(Y)

    def mul(self, X, Y):
        self.registers[X] = self.get_value(X) * self.get_value(Y)

    def mod(self, X, Y):
        self.registers[X] = self.get_value(X) % self.get_value(Y)

    def rcv(self, X):
        if self.get_value(X) != 0:
            return self.last_sound

    def jgz(self, X, Y):
        if self.get_value(X) > 0:
            self.instruction_pointer += self.get_value(Y) - 1

    def execute(self, instructions):
        while 0 <= self.instruction_pointer < len(instructions):
            instruction = instructions[self.instruction_pointer].split()
            cmd = instruction[0]
            args = instruction[1:]
            getattr(self, cmd)(*args)
            self.instruction_pointer += 1
            if cmd == 'rcv' and self.rcv(args[0]) is not None:
                print(f"Recovered frequency: {self.rcv(args[0])}")
                break

machine = SoundMachine()
machine.execute(data)

Recovered frequency: 3423


In [81]:
class SoundMachine:
    def __init__(self, pid, instructions):
        self.registers = {}
        self.last_sound = None
        self.instructions = instructions
        self.instruction_pointer = 0
        self.registers['p'] = pid
        self.queue = []

    def get_value(self, operand):
        try:
            return int(operand)
        except ValueError:
            return self.registers.get(operand, 0)

    def set(self, X, Y):
        self.registers[X] = self.get_value(Y)

    def add(self, X, Y):
        self.registers[X] = self.get_value(X) + self.get_value(Y)

    def mul(self, X, Y):
        self.registers[X] = self.get_value(X) * self.get_value(Y)

    def mod(self, X, Y):
        self.registers[X] = self.get_value(X) % self.get_value(Y)

    def snd(self, X):
        return self.get_value(X)

    def rcv(self, X):
        if len(self.queue) == 0:
          return False
        else:
          self.registers[X] = self.queue.pop(0)
          return True

    def jgz(self, X, Y):
        if self.get_value(X) > 0:
            self.instruction_pointer += self.get_value(Y) - 1

    def run(self):
        while True:
            instruction = self.instructions[self.instruction_pointer].split()
            cmd = instruction[0]
            args = instruction[1:]
            ret = getattr(self, cmd)(*args)
            if cmd == 'rcv' and not ret:
              return None
            self.instruction_pointer += 1
            if cmd == 'snd':
              return ret

m0 = SoundMachine(0, data)
m1 = SoundMachine(1, data)
count1 = 0
while True:
  r0 = m0.run()
  r1 = m1.run()
  if r0 == None and r1 == None:
    print("deadlock")
    break
  if r0 != None:
    m1.queue.append(r0)
  if r1 != None:
    count1 += 1
    m0.queue.append(r1)
count1

deadlock


7493